In [1]:
import pandas as pd
import numpy as np

1\. **Text files**

Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named `data_int.txt`. Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named `data_float.txt`. Use the `cat` command to print the content of the file.
+ load the `txt` file of the previous point and convert it to a `csv` file by hand.

In [60]:
out_file_name = "data/data_int.txt"
a = str(np.random.randint(100, size = 5))
print(a)

with open(out_file_name, 'w') as outfile:
    outfile.write(a)

!type data\data_int.txt


out_file_name2 = "data\data_float.txt"
b = str(np.random.rand(5,5))
print(b)

with open(out_file_name2, 'w') as outfile:
    outfile.write(b)

!type data\data_float.txt

out_file_name3 = "data\data_float.csv"
with open(out_file_name2, 'r') as infile:
    with open(out_file_name3, 'w') as outfile:

        for line in infile:
            temp = line.replace('[','').replace(']','')
            outfile.write(','.join(temp.split()) + '\n')
    
!type data\data_float.csv
    

[49 28  4 59 85]
[49 28  4 59 85]
[[0.81395302 0.9732212  0.97235308 0.71993355 0.00147064]
 [0.91080595 0.16205656 0.27333068 0.92593843 0.05495631]
 [0.41550259 0.44702059 0.28161638 0.94878248 0.27236277]
 [0.47472741 0.16378137 0.52373216 0.62417455 0.01584173]
 [0.22501811 0.37266248 0.33215983 0.85307816 0.17198639]]
[[0.81395302 0.9732212  0.97235308 0.71993355 0.00147064]
 [0.91080595 0.16205656 0.27333068 0.92593843 0.05495631]
 [0.41550259 0.44702059 0.28161638 0.94878248 0.27236277]
 [0.47472741 0.16378137 0.52373216 0.62417455 0.01584173]
 [0.22501811 0.37266248 0.33215983 0.85307816 0.17198639]]
0.81395302,0.9732212,0.97235308,0.71993355,0.00147064
0.91080595,0.16205656,0.27333068,0.92593843,0.05495631
0.41550259,0.44702059,0.28161638,0.94878248,0.27236277
0.47472741,0.16378137,0.52373216,0.62417455,0.01584173
0.22501811,0.37266248,0.33215983,0.85307816,0.17198639


2\. **JSON files**

Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [98]:
df = pd.read_json('data/user_data.json')
df = df[df['CreditCardType']=='American Express']

out_file_name = "data/user_data_American_Express.csv"

with open(out_file_name, 'w') as outfile:
    outfile.write(df.to_csv(index=False))

!type data\user_data_American_Express.csv

ID,JobTitle,EmailAddress,FirstNameLastName,CreditCard,CreditCardType

2,Investment  Advisor,Clint_Thorpe5003@bulaffy.com,Clint Thorpe,7083-8766-0251-2345,American Express

12,Retail Trainee,Phillip_Carpenter9505@famism.biz,Phillip Carpenter,3657-0088-0820-5247,American Express

28,Project Manager,Russel_Graves1378@extex.org,Russel Graves,6718-4818-8011-6024,American Express

39,Stockbroker,Leanne_Newton1268@typill.biz,Leanne Newton,5438-0816-4166-4847,American Express

57,Budget Analyst,Tony_Giles1960@iatim.tech,Tony Giles,8130-3425-7573-7745,American Express

62,CNC Operator,Owen_Allcott5125@bauros.biz,Owen Allcott,4156-0107-7210-2630,American Express

68,Project Manager,Liam_Lynn3280@kideod.biz,Liam Lynn,7152-3247-6053-2233,American Express

74,Dentist,Regina_Woodcock5820@yahoo.com,Regina Woodcock,0208-1753-3870-8002,American Express

81,HR Specialist,Carter_Wallace9614@atink.com,Carter Wallace,4256-7201-6717-4322,American Express

92,Staffing Consultant,Maia_Stark2797@jiman.org,Maia

3\. **CSV files with Pandas**

Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

4\. **Reading the credit card numbers**

Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

5\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

In [ ]:
from IPython.display import Image
Image("images/data_format.png")

*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, ``pack'' the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.

b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_OSEMN.ipynb`, and verify that the content of the `txt` and binary files is consistent.

c) What is the difference of the size on disk between equivalent `txt` and binary files?